In [2]:
from pathlib import Path

In [3]:
Path.cwd()

PosixPath('/home/windact/cloud_detection/notebooks')

In [13]:
Path.cwd().parent.resolve()

PosixPath('/home/windact/cloud_detection')

In [4]:
# __init__.py

In [16]:
PACKAGE_DIR = Path.cwd().parent.resolve() / "cloudnet-package"
PACKAGE_DIR

PosixPath('/home/windact/cloud_detection/cloudnet-package')

In [17]:
TRAINER_DIR = PACKAGE_DIR / "trainer"
TRAINER_DIR

PosixPath('/home/windact/cloud_detection/cloudnet-package/trainer')

In [19]:
if not TRAINER_DIR.exists():
    TRAINER_DIR.mkdir(exist_ok= True)

In [20]:
!touch /home/windact/cloud_detection/cloudnet-package/trainer/__init__.py

In [49]:
# requirements.txt

In [53]:
%%writefile /home/windact/cloud_detection/cloudnet-package/requirements.txt
tensorflow>=2.6.2,<2.7.0
tensorflow-addons>=0.14.0,<0.15.0
tensorflow-io>=0.21.0,<0.22.0

Overwriting /home/windact/cloud_detection/cloudnet-package/requirements.txt


In [38]:
%%writefile /home/windact/cloud_detection/cloudnet-package/setup.py
from setuptools import find_packages
from setuptools import setup
import os
from pathlib import Path

# Packages that are required for this module to be executed
#here = os.path.abspath(os.path.dirname(__file__))
#lib_path = Path.cwd().resolve() / 'requirements.txt'

# def list_reqs(fname="f.txt"):
#     with open(fname) as fd:
#         return fd.read().splitlines()

# with open("requirements.txt", "r") as f:
#     rq = f.read().splitlines()

# REQUIRED_PACKAGES = ['pandas<=1.3.5','tensorflow<=2.6.2','tensorflow-io<=0.23.1','tensorflow-addons<=0.15.0']

REQUIRED_PACKAGES = ['pandas>=1.1.5,<1.2.0','tensorflow>=2.6.2,<2.7.0','tensorflow-addons>=0.14.0,<0.15.0','tensorflow-io>=0.21.0,<0.22.0']

#print("*-*-*-*-*-*-*-*-HERE*-*-*-*-*-*-*---*")
#here = os.path.abspath(os.path.dirname(__file__))
#print(lib_path)
#REQUIRED_PACKAGES = ['some_PyPI_package>=1.0']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)


Writing /home/windact/cloud_detection/cloudnet-package/setup.py


In [40]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization,\
#     Activation, Dropout
# from tensorflow import  keras

# from pathlib import Path
# import argparse
# import sys
# import logging

# from datetime import datetime
# import pandas as pd

# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping,TensorBoard

# from tensorflow.keras import backend as K
# from tensorflow import keras

# import tensorflow_addons as tfa
# import tensorflow_io as tfio

In [41]:
# tensorflow
# pandas
# tensorflow_addons
# tensorflow_io

In [42]:
# import tensorflow as tf 
# import tensorflow_addons as tfa
# import tensorflow_io as tfio
# import pandas as pd

In [43]:
# print(pd.__version__)

In [44]:
# pipeline.py

In [32]:
%%writefile /home/windact/cloud_detection/cloudnet-package/trainer/pipeline.py
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio

IMG_HEIGHT = tf.constant(192, dtype=tf.int32)
IMG_WIDTH = tf.constant(192,dtype=tf.int32)
EPOCHS  = 2

@tf.function
def load_image(img_path):
    image = tf.io.read_file(img_path)
    image = tfio.experimental.image.decode_tiff(image)
    
    return image

@tf.function
def parser(row_csv):
    # decoding the csv file
    chip_id,B02_path,B03_path,B04_path,B08_path,label_path = tf.io.decode_csv(records = row_csv, record_defaults = ["chip_id","B02_path","B03_path","B04_path","B08_path","label_path"], field_delim=';')
    
    B02_img = load_image(B02_path)
    # B02_img = B02_img[:,:,0]
    # B02_img = tf.expand_dims(B02_img, axis = -1, name=None)
    
    B03_img = load_image(B03_path)
    # B03_img = B03_img[:,:,0]
    # B03_img = tf.expand_dims(B03_img, axis = -1, name=None)
    
    B04_img = load_image(B04_path)
    # B04_img = B04_img[:,:,0]
    # B04_img = tf.expand_dims(B04_img, axis = -1, name=None)
    
    B08_img = load_image(B08_path)
    # B08_img = B08_img[:,:,0]
    # B08_img = tf.expand_dims(B08_img, axis = -1, name=None)
    
    image = tf.concat([B02_img,B03_img,B04_img,B08_img], axis = -1)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32, saturate=False)
    
    label = load_image(label_path)
    # label = label[:,:,0]
    # label = tf.expand_dims(label, axis = -1, name=None)
    
    label = tf.image.convert_image_dtype(label, dtype=tf.float32, saturate=False)
    
    
    return image, label

@tf.function
def img_reshape(img,msk):
    img_o = tf.image.resize(img,(192, 192),preserve_aspect_ratio=False)
    msk_o = tf.image.resize(msk,(192, 192),preserve_aspect_ratio=False)
    
    return img_o,msk_o
    

@tf.function
def rotate_clk_img_and_msk(img, msk):
    angles_tensor = tf.constant([4, 6, 8, 10, 12, 14, 16, 18, 20], dtype=tf.float32)
    angle = tf.random.shuffle(angles_tensor)[0]
    # Image
    img_o = tfa.image.rotate(images = img,angles = angle,interpolation = "nearest",fill_mode = "reflect",fill_value = 0.0)
    # Label
    msk_o = tfa.image.rotate(images = msk,angles = angle,interpolation = "nearest",fill_mode = "reflect",fill_value = 0.0)
    
    return img_o, msk_o

@tf.function
def rotate_cclk_img_and_msk(img, msk):
    angles_tensor = tf.constant([-20, -18, -16, -14, -12, -10, -8, -6, -4],dtype=tf.float32)
    angle = tf.random.shuffle(angles_tensor)[0]
    # Image
    img_o = tfa.image.rotate(images = img,angles = angle,interpolation = "nearest",fill_mode = "reflect",fill_value = 0.0)
    # Label
    msk_o = tfa.image.rotate(images = msk,angles = angle,interpolation = "nearest",fill_mode = "reflect",fill_value = 0.0)
    
    return img_o, msk_o

@tf.function
def flipping_img_and_msk(img, msk):
    img_o = tf.image.flip_left_right(img)
    img_o = tf.image.flip_up_down(img)
    
    msk_o = tf.image.flip_left_right(msk)
    msk_o = tf.image.flip_up_down(msk)
    
    return img_o,msk_o


    
@tf.function
def zoom_img_and_msk(img, msk,height = 512,width = 512):

    zoom_factor_tensor = tf.constant([1.2, 1.5, 1.8, 2, 2.2, 2.5], dtype=tf.float32)  # currently doesn't have zoom out!
    zoom_factor = tf.random.shuffle(zoom_factor_tensor)[0]
    # print("*-*-*-*-*-*")
    # print(img.shape)
    #h,w,c = img.shape
    # h = height
    # w = width
    h = tf.cast(height,dtype= tf.int32)
    w = tf.cast(width,dtype= tf.int32)
    
    # img = tf.cast(img, dtype=tf.float32)
    # msk = tf.cast(msk, dtype=tf.float32)
    

    # width and height of the zoomed image
    zh = tf.math.multiply(zoom_factor, tf.cast(h, dtype=tf.float32))
    zh = tf.cast(zh,dtype= tf.int32)
    
    zw = tf.math.multiply(zoom_factor, tf.cast(w, dtype=tf.float32))
    zw = tf.cast(zw,dtype= tf.int32)
    
    # zh = int(np.round(zoom_factor * h))
    # zw = int(np.round(zoom_factor * w))

    img = tf.image.resize(img,(zh, zw),preserve_aspect_ratio=False)
    msk = tf.image.resize(msk,(zh, zw),preserve_aspect_ratio=False)
    
    region_tensor = tf.constant([0, 1, 2, 3, 4],dtype= tf.float32)
    region = tf.random.shuffle(region_tensor)[0]

    # zooming out
    # tf.print("before zoom")
    # tf.print(img.dtype)
    
    if tf.math.less_equal(zoom_factor, tf.constant(1, dtype= tf.float32)):
        outimg = img
        outmsk = msk
        
        # tf.print("zoom 1")
        # tf.print(img.dtype)

    # zooming in
    # else:
    #     # Initializing
    #     outimg = tf.zeros_like(img, dtype=tf.float32, name=None)
    #     outmsk = tf.zeros_like(msk, dtype=tf.float32, name=None)
        # bounding box of the clipped region within the input array
    elif tf.math.equal(region, tf.constant(0,dtype= tf.float32)):
        outimg = img[0:h, 0:w,:]
        outmsk = msk[0:h, 0:w,:]
        
        # tf.print("zoom 0")
        # tf.print(img.dtype)
        
    elif tf.math.equal(region, tf.constant(1,dtype= tf.float32)):
        outimg = img[0:h, zw - w:zw,:]
        outmsk = msk[0:h, zw - w:zw,:]
        
        # tf.print("zoom 11")
        # tf.print(img.dtype)
        
    elif tf.math.equal(region, tf.constant(2,dtype= tf.float32)):
        outimg = img[zh - h:zh, 0:w,:]
        outmsk = msk[zh - h:zh, 0:w,:]
        
        # tf.print("zoom 2")
        # tf.print(img.dtype)
        
    elif tf.math.equal(region, tf.constant(3,dtype= tf.float32)):
        outimg = img[zh - h:zh, zw - w:zw,:]
        outmsk = msk[zh - h:zh, zw - w:zw,:]
        
        # tf.print("zoom 3")
        # tf.print(img.dtype)
        
    # if tf.math.equal(region, tf.constant(4,dtype= tf.float32)):

    else:
        # tf.print("zoom 4")
        # tf.print(img.dtype)
        
        marh = tf.math.floordiv( h, tf.constant(2))
        marw = tf.math.floordiv( w, tf.constant(2))

        zh_div = tf.math.floordiv( zh, tf.constant(2))
        zw_div = tf.math.floordiv( zw, tf.constant(2))

        zh_div_add = tf.math.add( zh_div, marh)
        zh_div_minus = tf.math.subtract( zh_div, marh)

        zw_div_add = tf.math.add( zw_div, marw)
        zw_div_minus = tf.math.subtract( zw_div, marw)

        outimg = img[zh_div_minus:zw_div_add, zw_div_minus:zw_div_add,:]
        outmsk = msk[zh_div_minus:zw_div_add, zw_div_minus:zw_div_add,:]

        # outimg = img[(zh // 2 - marh):(zh // 2 + marh), (zw // 2 - marw):(zw // 2 + marw),:]
        # outmsk = msk[(zh // 2 - marh):(zh // 2 + marh), (zw // 2 - marw):(zw // 2 + marw),:]

    # to make sure the output is in the same size of the input
    img_o = tf.image.resize(outimg,(h, w),preserve_aspect_ratio=False)
    msk_o = tf.image.resize(outmsk,(h, w),preserve_aspect_ratio=False)
    return img_o, msk_o


@tf.function
def data_augmentation(img, msk):
    
    coin_rotate_clk_img_and_msk = tf.random.uniform(shape = (1,1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
    coin_rotate_cclk_img_and_msk = tf.random.uniform(shape = (1,1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
    coin_flipping_img_and_msk = tf.random.uniform(shape = (1,1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
    coin_zoom_img_and_msk = tf.random.uniform(shape = (1,1), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)

    # rotate_clk_img_and_msk
    if tf.math.greater_equal(coin_rotate_clk_img_and_msk, tf.constant(0.5)):
        # tf.print(tf.constant("rotate_clk_img_and_msk"))
        # tf.print(coin_rotate_clk_img_and_msk)
        img,msk = rotate_clk_img_and_msk(img, msk)
    
    # rotate_cclk_img_and_msk
    if tf.math.greater_equal(coin_rotate_cclk_img_and_msk, tf.constant(0.5)):
        # tf.print(tf.constant("rotate_cclk_img_and_msk"))
        # tf.print(coin_rotate_cclk_img_and_msk)
        img,msk = rotate_cclk_img_and_msk(img, msk)
        
    # flipping_img_and_msk
    # rotate_cclk_img_and_msk
    if tf.math.greater_equal(coin_flipping_img_and_msk, tf.constant(0.5)):
        # tf.print(tf.constant("flipping_img_and_msk"))
        # tf.print(coin_flipping_img_and_msk)
        img,msk = flipping_img_and_msk(img, msk)
    
    # zoom_img_and_msk
    if tf.math.greater_equal(coin_zoom_img_and_msk, tf.constant(0.5)):
        # tf.print(tf.constant("coin_zoom_img_and_msk"))
        # tf.print(coin_zoom_img_and_msk)
        img,msk = zoom_img_and_msk(img, msk,height = IMG_HEIGHT,width = IMG_WIDTH)
    
    return img,msk

def load_dataset(file_paths, reshape = False,buffer_size = 1000, batch_size = 12,training = True, num_epochs = EPOCHS):
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    
    dataset = tf.data.TextLineDataset(filenames=file_paths)
    dataset = dataset.skip(1).map(parser).cache()
    
    # reshape
    if reshape == True:
        dataset = dataset.map(img_reshape)
    
    dataset = dataset.map(data_augmentation)
    
    if training == True:
        dataset = dataset.shuffle(buffer_size =buffer_size)
        dataset = dataset.repeat(count=num_epochs)
    else:
        dataset = dataset.repeat(count=1)
        
    dataset = dataset.batch(batch_size = batch_size,drop_remainder = False).prefetch(buffer_size=AUTOTUNE)
    
    return dataset
    



Overwriting /home/windact/cloud_detection/cloudnet-package/trainer/pipeline.py


In [24]:
# utils.py

In [28]:
%%writefile /home/windact/cloud_detection/cloudnet-package/trainer/utils.py
from tensorflow.keras import backend as K
from tensorflow import keras

smooth = 0.0000001


def jacc_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))


class ADAMLearningRateTracker(keras.callbacks.Callback):
    """It prints out the last used learning rate after each epoch (useful for resuming a training)
    original code: https://github.com/keras-team/keras/issues/7874#issuecomment-329347949
    """

    def __init__(self, end_lr):
        super(ADAMLearningRateTracker, self).__init__()
        self.end_lr = end_lr

    def on_epoch_end(self, epoch, logs={}):  # works only when decay in optimizer is zero
        optimizer = self.model.optimizer
        # t = K.cast(optimizer.iterations, K.floatx()) + 1
        # lr_t = K.eval(optimizer.lr * (K.sqrt(1. - K.pow(optimizer.beta_2, t)) /
        #                               (1. - K.pow(optimizer.beta_1, t))))
        # print('\n***The last Actual Learning rate in this epoch is:', lr_t,'***\n')
        print('\n***The last Basic Learning rate in this epoch is:', K.eval(optimizer.lr), '***\n')
        # stops the training if the basic lr is less than or equal to end_learning_rate
        if K.eval(optimizer.lr) <= self.end_lr:
            print("training is finished")
            self.model.stop_training = True

Overwriting /home/windact/cloud_detection/cloudnet-package/trainer/utils.py


In [29]:
# model.py

In [30]:
%%writefile /home/windact/cloud_detection/cloudnet-package/trainer/model.py
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization,\
    Activation, Dropout
from tensorflow import  keras

"""
Some parts borrowed from https://www.kaggle.com/cjansen/u-net-in-keras
"""


def bn_relu(input_tensor):
    """It adds a Batch_normalization layer before a Relu
    """
    input_tensor = BatchNormalization(axis=3)(input_tensor)
    return Activation("relu")(input_tensor)


def contr_arm(input_tensor, filters, kernel_size):
    """It adds a feedforward signal to the output of two following conv layers in contracting path
       TO DO: remove keras.layers.add and replace it with add only
    """

    x = Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = bn_relu(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = bn_relu(x)

    filters_b = filters // 2
    kernel_size_b = (kernel_size[0]-2, kernel_size[0]-2)  # creates a kernl size of (1,1) out of (3,3)

    x1 = Conv2D(filters_b, kernel_size_b, padding='same')(input_tensor)
    x1 = bn_relu(x1)

    x1 = concatenate([input_tensor, x1], axis=3)
    x = keras.layers.add([x, x1])
    x = Activation("relu")(x)
    return x


def imprv_contr_arm(input_tensor, filters, kernel_size ):
    """It adds a feedforward signal to the output of two following conv layers in contracting path
    """

    x = Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = bn_relu(x)

    x0 = Conv2D(filters, kernel_size, padding='same')(x)
    x0 = bn_relu(x0)

    x = Conv2D(filters, kernel_size, padding='same')(x0)
    x = bn_relu(x)

    filters_b = filters // 2
    kernel_size_b = (kernel_size[0]-2, kernel_size[0]-2)  # creates a kernl size of (1,1) out of (3,3)

    x1 = Conv2D(filters_b, kernel_size_b, padding='same')(input_tensor)
    x1 = bn_relu(x1)

    x1 = concatenate([input_tensor, x1], axis=3)

    x2 = Conv2D(filters, kernel_size_b, padding='same')(x0)
    x2 = bn_relu(x2)

    x = keras.layers.add([x, x1, x2])
    x = Activation("relu")(x)
    return x


def bridge(input_tensor, filters, kernel_size):
    """It is exactly like the identity_block plus a dropout layer. This block only uses in the valley of the UNet
    """

    x = Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = bn_relu(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = Dropout(.15)(x)
    x = bn_relu(x)

    filters_b = filters // 2
    kernel_size_b = (kernel_size[0]-2, kernel_size[0]-2)  # creates a kernl size of (1,1) out of (3,3)

    x1 = Conv2D(filters_b, kernel_size_b, padding='same')(input_tensor)
    x1 = bn_relu(x1)

    x1 = concatenate([input_tensor, x1], axis=3)
    x = keras.layers.add([x, x1])
    x = Activation("relu")(x)
    return x


def conv_block_exp_path(input_tensor, filters, kernel_size):
    """It Is only the convolution part inside each expanding path's block
    """

    x = Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = bn_relu(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = bn_relu(x)
    return x


def conv_block_exp_path3(input_tensor, filters, kernel_size):
    """It Is only the convolution part inside each expanding path's block
    """

    x = Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = bn_relu(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = bn_relu(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = bn_relu(x)
    return x


def add_block_exp_path(input_tensor1, input_tensor2, input_tensor3):
    """It is for adding two feed forwards to the output of the two following conv layers in expanding path
    """

    x = keras.layers.add([input_tensor1, input_tensor2, input_tensor3])
    x = Activation("relu")(x)
    return x


def improve_ff_block4(input_tensor1, input_tensor2 ,input_tensor3, input_tensor4, pure_ff):
    """It improves the skip connection by using previous layers feature maps
       TO DO: shrink all of ff blocks in one function/class
    """

    for ix in range(1):
        if ix == 0:
            x1 = input_tensor1
        x1 = concatenate([x1, input_tensor1], axis=3)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)

    for ix in range(3):
        if ix == 0:
            x2 = input_tensor2
        x2 = concatenate([x2, input_tensor2], axis=3)
    x2 = MaxPooling2D(pool_size=(4, 4))(x2)

    for ix in range(7):
        if ix == 0:
            x3 = input_tensor3
        x3 = concatenate([x3, input_tensor3], axis=3)
    x3 = MaxPooling2D(pool_size=(8, 8))(x3)

    for ix in range(15):
        if ix == 0:
            x4 = input_tensor4
        x4 = concatenate([x4, input_tensor4], axis=3)
    x4 = MaxPooling2D(pool_size=(16, 16))(x4)

    x = keras.layers.add([x1, x2, x3, x4, pure_ff])
    x = Activation("relu")(x)
    return x


def improve_ff_block3(input_tensor1, input_tensor2, input_tensor3, pure_ff):
    """It improves the skip connection by using previous layers feature maps
    """

    for ix in range(1):
        if ix == 0:
            x1 = input_tensor1
        x1 = concatenate([x1, input_tensor1], axis=3)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)

    for ix in range(3):
        if ix == 0:
            x2 = input_tensor2
        x2 = concatenate([x2, input_tensor2], axis=3)
    x2 = MaxPooling2D(pool_size=(4, 4))(x2)

    for ix in range(7):
        if ix == 0:
            x3 = input_tensor3
        x3 = concatenate([x3, input_tensor3], axis=3)
    x3 = MaxPooling2D(pool_size=(8, 8))(x3)

    x = keras.layers.add([x1, x2, x3, pure_ff])
    x = Activation("relu")(x)
    return x


def improve_ff_block2(input_tensor1, input_tensor2, pure_ff):
    """It improves the skip connection by using previous layers feature maps
    """

    for ix in range(1):
        if ix == 0:
            x1 = input_tensor1
        x1 = concatenate([x1, input_tensor1], axis=3)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)

    for ix in range(3):
        if ix == 0:
            x2 = input_tensor2
        x2 = concatenate([x2, input_tensor2], axis=3)
    x2 = MaxPooling2D(pool_size=(4, 4))(x2)

    x = keras.layers.add([x1, x2, pure_ff])
    x = Activation("relu")(x)
    return x


def improve_ff_block1(input_tensor1, pure_ff):
    """It improves the skip connection by using previous layers feature maps
    """

    for ix in range(1):
        if ix == 0:
            x1 = input_tensor1
        x1 = concatenate([x1, input_tensor1], axis=3)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)

    x = keras.layers.add([x1, pure_ff])
    x = Activation("relu")(x)
    return x


def model_arch(input_rows=192, input_cols=192, num_of_channels=4, num_of_classes=1):
    inputs = Input((input_rows, input_cols, num_of_channels))
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)

    conv1 = contr_arm(conv1, 32, (3, 3))
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = contr_arm(pool1, 64, (3, 3))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = contr_arm(pool2, 128, (3, 3))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = contr_arm(pool3, 256, (3, 3))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = imprv_contr_arm(pool4, 512, (3, 3))
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = bridge(pool5, 1024, (3, 3))

    convT7 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv6)
    prevup7 = improve_ff_block4(input_tensor1=conv4, input_tensor2=conv3, input_tensor3=conv2, input_tensor4=conv1, pure_ff=conv5)
    up7 = concatenate([convT7, prevup7], axis=3)
    conv7 = conv_block_exp_path3(input_tensor=up7, filters=512, kernel_size=(3, 3))
    conv7 = add_block_exp_path(conv7, conv5, convT7)

    convT8 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv7)
    prevup8 = improve_ff_block3(input_tensor1=conv3, input_tensor2=conv2, input_tensor3=conv1, pure_ff=conv4)
    up8 = concatenate([convT8, prevup8], axis=3)
    conv8 = conv_block_exp_path(input_tensor=up8, filters=256, kernel_size=(3, 3))
    conv8 = add_block_exp_path(input_tensor1=conv8, input_tensor2=conv4, input_tensor3=convT8)

    convT9 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv8)
    prevup9 = improve_ff_block2(input_tensor1=conv2, input_tensor2=conv1, pure_ff=conv3)
    up9 = concatenate([convT9, prevup9], axis=3)
    conv9 = conv_block_exp_path(input_tensor=up9, filters=128, kernel_size=(3, 3))
    conv9 = add_block_exp_path(input_tensor1=conv9, input_tensor2=conv3, input_tensor3=convT9)

    convT10 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv9)
    prevup10 = improve_ff_block1(input_tensor1=conv1, pure_ff=conv2)
    up10 = concatenate([convT10, prevup10], axis=3)
    conv10 = conv_block_exp_path(input_tensor=up10, filters=64, kernel_size=(3, 3))
    conv10 = add_block_exp_path(input_tensor1=conv10, input_tensor2=conv2, input_tensor3=convT10)

    convT11 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv10)
    up11 = concatenate([convT11, conv1], axis=3)
    conv11 = conv_block_exp_path(input_tensor=up11, filters=32, kernel_size=(3, 3))
    conv11 = add_block_exp_path(input_tensor1=conv11, input_tensor2=conv1, input_tensor3=convT11)

    conv12 = Conv2D(num_of_classes, (1, 1), activation='sigmoid')(conv11)

    return Model(inputs=[inputs], outputs=[conv12])


Writing /home/windact/cloud_detection/cloudnet-package/trainer/model.py


In [51]:
# task.py

In [52]:
%%writefile /home/windact/cloud_detection/cloudnet-package/trainer/task.py
from pathlib import Path
import argparse
import sys
import logging

from datetime import datetime

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping,TensorBoard

from trainer.utils import ADAMLearningRateTracker, jacc_coef
from trainer.model import model_arch

# logger
model_logger = logging.getLogger(__name__)
model_logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

model_logger_file_handler = logging.FileHandler('model.log')
model_logger_file_handler.setFormatter(formatter)

model_logger.addHandler(model_logger_file_handler)



def _parse_arguments(argv):
    """Parses command-line arguments."""
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--train_data_path',
        help='train data path',
        type=str, default="/home/jupyter/cloud_detection/data/train_data.csv")
    parser.add_argument(
        '--val_data_path',
        help='validation data path',
        type=str, default="/home/jupyter/cloud_detection/data/val_data.csv")
    parser.add_argument(
        '--batch_size',
        help='model batch size',
        type=int, default=12)
    parser.add_argument(
        '--epochs',
        help='The number of epochs to train',
        type=int, default=10)
    parser.add_argument(
        '--random_state',
        help='random state',
        type=int, default=42)
    parser.add_argument(
        '--starting_learning_rate',
        help='starting learning rate',
        type=float, default=1e-4)
    parser.add_argument(
        '--end_learning_rate',
        help='end learning rate',
        type=float, default=1e-8)
    parser.add_argument(
        '--input_rows',
        help='input image input_rows',
        type=int, default=192)
    parser.add_argument(
        '--input_cols',
        help='input image input_rows',
        type=int, default=192)
    parser.add_argument(
        '--patience',
        help='patience for early_s_patience.ReduceLROnPlateau',
        type=int, default=15)
    parser.add_argument(
        '--decay_factor',
        help='decay_factor for tensorflow.keras.callbacks.ReduceLROnPlateau',
        type=float, default=0.7)
    parser.add_argument(
        '--experiment_name',
        help='experiment_name',
        type=str, default="cloudnet")
    parser.add_argument(
        '--early_s_patience',
        help='tensorflow.keras.callbacks.EarlyStopping patience',
        type=int, default=20)
    parser.add_argument(
        '--num_of_channels',
        help='num_of_channels',
        type=int, default=16)
    parser.add_argument(
        '--num_of_classes',
        help='num_of_classes',
        type=int, default=4)
    parser.add_argument(
        '--reshape',
        help='reshape image and mask to the sampe shape',
        type=bool, default=True)
    parser.add_argument(
        '--quick_test',
        help='run the model on a smaler sample',
        type=bool, default=False)
    parser.add_argument(
        '--train_resume',
        help='resume train or not',
        type=bool, default=False)
    parser.add_argument(
        '--job-dir',
        help='Directory where to save the given model',
        type=str, default='cloud_detection_models/')
    
    return parser.parse_known_args(argv)

def main():
    
    # Get the arguments
    args = _parse_arguments(sys.argv[1:])[0]

    #BATCH_SIZE = args.batch_size
    # SHUFFLE_BUFFER = 10 * BATCH_SIZE
    # RANDOM_STATE = args.random_state
    # AUTOTUNE = tf.data.experimental.AUTOTUNE
    TRAIN_DATA_PATH = args.train_data_path
    VAL_DATA_PATH = args.val_data_path

    #quick_test = args.quick_test
    current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
    experiment_name = f"{args.experiment_name}_{current_time}"
    
    ROOT_DIR = Path.cwd().resolve()
    MODEL_DIR = ROOT_DIR / "models"
    TRAIN_DIR = MODEL_DIR / "train"
    TEST_DIR = MODEL_DIR / "test"
    EXP_DIR = TRAIN_DIR / experiment_name
    
    ORIGINAL_MODEL_WEIGHT_PATH = (MODEL_DIR / "original_weights") / "Cloud-Net_trained_on_38-Cloud_training_patches.h5" # not implemented

    folders = [MODEL_DIR,TRAIN_DIR,TEST_DIR,EXP_DIR]
    for folder in folders:
        if not folder.exists():
            folder.mkdir(parents = False,exist_ok= True)

    MODEL_WEIGHTS_PATH = ROOT_DIR/"model_weights"
    if not MODEL_WEIGHTS_PATH.exists():
        MODEL_WEIGHTS_PATH.mkdir()

    weights_path = MODEL_WEIGHTS_PATH / "weights.{epoch:02d}-{val_loss:.2f}.hdf5"

    random_state = args.random_state

    # hparams
    # starting_learning_rate = args.starting_learning_rate
    # end_learning_rate = args.end_learning_rate
    # epochs = args.epochs # just a huge number. The actual training should not be limited by this value
    # #val_ratio = 0.2
    # patience = args.patience
    # decay_factor = args.decay_factor
    # experiment_name = args.experiment_name
    # early_s_patience = args.early_s_patience

    # params
    input_rows = args.input_rows
    input_cols = args.input_cols
    # img_shape = (input_rows,input_cols)
    num_of_channels = args.num_of_channels
    num_of_classes = args.num_of_classes
    reshape = args.reshape

    # hparams
    batch_size = args.batch_size
    starting_learning_rate = args.starting_learning_rate
    end_learning_rate = args.end_learning_rate
    max_num_epochs = args.epochs # just a huge number. The actual training should not be limited by this value
    patience = args.patience
    decay_factor = args.decay_factor

    early_s_patience = args.early_s_patience
    train_resume = args.train_resume
    
    # log
    model_logger.info("All parameters have been paresed")
    
    # datasets
    train_dataset = load_dataset(file_paths= TRAIN_DATA_PATH, training = True,reshape= reshape, num_epochs=max_num_epochs)
    val_dataset = load_dataset(file_paths= VAL_DATA_PATH, training = False,reshape= reshape)

    # Model
    strategy = tf.distribute.MirroredStrategy()
    model_logger.info('Number of devices: {}'.format(strategy.num_replicas_in_sync))

    with strategy.scope():
        model = model_arch(input_rows=input_rows,
                                           input_cols=input_cols,
                                           num_of_channels=num_of_channels,
                                           num_of_classes=num_of_classes)
        
        model.compile(optimizer=Adam(learning_rate=starting_learning_rate), loss=jacc_coef, metrics=[jacc_coef])
    # model.summary()

    model_checkpoint = ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True)
    lr_reducer = ReduceLROnPlateau(factor=decay_factor, cooldown=0, patience=patience, min_lr=end_learning_rate, verbose=1)
    csv_logger = CSVLogger(EXP_DIR / '_log_1.log')
    tensorboard = TensorBoard(log_dir= EXP_DIR / 'logs', histogram_freq=0, write_graph=True,write_images=False, write_steps_per_second=False,
                                   update_freq='epoch',profile_batch=0, embeddings_freq=0, embeddings_metadata=None, **kwargs)

    if train_resume:
        model.load_weights(ORIGINAL_MODEL_WEIGHT_PATH)
         model_logger.info("\nTraining resumed...")
    else:
         model_logger.info("\nTraining started from scratch... ")

    model_logger("Experiment name: ", experiment_name)
    model_logger("Input image size: ", (input_rows, input_cols))
    model_logger("Number of input spectral bands: ", num_of_channels)
    model_logger("Learning rate: ", starting_learning_rate)
    model_logger("# Epochs: ", max_num_epochs)
    model_logger("Batch size: ", batch_size, "\n")
    
    model.fit(train_dataset,validation_data = val_dataset,epochs = max_num_epochs,verbose = 1,
             callbacks=[model_checkpoint, lr_reducer, ADAMLearningRateTracker(end_learning_rate), csv_logger,tensorboard])
    
if __name__ == '__main__':
    main()

Overwriting /home/windact/cloud_detection/cloudnet-package/trainer/task.py


In [ ]:
# in_rows = args.input_rows
# in_cols = args.input_cols
# num_of_channels = args.num_of_channels
# num_of_classes = args.num_of_classes
# starting_learning_rate = args.starting_learning_rate
# end_learning_rate = args.end_learning_rate
# max_num_epochs = 2  # just a huge number. The actual training should not be limited by this value
# patience = args.patience
# decay_factor = args.decay_factor
# batch_size = args.batch_size
# early_s_patience = args.early_s_patience
# current_time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# experiment_name = f"{args.experiment_name}_{current_time}"
# # max_bit = 65535  # maximum gray level in landsat 8 images
# weights_path = EXP_DIR / "weights.{epoch:02d}-{val_loss:.2f}.hdf5"
# train_resume = False

